
---



by Tejasv Dua
21BCI0251


In [ ]:
!pip install sktime
!pip install gradio


  Using cached sktime-0.37.0-py3-none-any.whl.metadata (34 kB)
Using cached sktime-0.37.0-py3-none-any.whl (37.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00


In [ ]:
# !pip install pmdarima
# !pip install --user pmdarima

In [ ]:
# !pip install pandas matplotlib seaborn scikit-learn sktime pmdarima gradio


In [ ]:
!pip uninstall -y numpy pmdarima statsmodels scipy
!pip install numpy==1.24.3 scipy==1.10.1 statsmodels==0.13.5 pmdarima==2.0.4


Found existing installation: numpy 1.24.3
Uninstalling numpy-1.24.3:
  Successfully uninstalled numpy-1.24.3
Found existing installation: pmdarima 2.0.4
Uninstalling pmdarima-2.0.4:
  Successfully uninstalled pmdarima-2.0.4
Found existing installation: statsmodels 0.13.5
Uninstalling statsmodels-0.13.5:
  Successfully uninstalled statsmodels-0.13.5
Found existing installation: scipy 1.10.1
Uninstalling scipy-1.10.1:
  Successfully uninstalled scipy-1.10.1
  Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.6 kB)
  Using cached scipy-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
  Using cached statsmodels-0.13.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.5 kB)
  Using cached pmdarima-2.0.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_28_x86_64.whl.metadata (7.8 kB)
Using cached numpy-1.24.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 

# Time series forecasting and evaluation

In [ ]:
import numpy
import pmdarima
import statsmodels
print("numpy:", numpy.__version__)
print("pmdarima:", pmdarima.__version__)
print("statsmodels:", statsmodels.__version__)


numpy: 1.24.3
pmdarima: 2.0.4
statsmodels: 0.13.5


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.base import ForecastingHorizon
import gradio as gr

# Load dataset

In [ ]:
# Load and preprocess data
df = pd.read_csv("/content/Consumption Dataset - Dataset (1).csv")
df['Date Time Served'] = pd.to_datetime(df['Date Time Served'])
df['Date'] = df['Date Time Served'].dt.date

# Aggregate daily consumption per bar-brand pair


In [ ]:
# Aggregate daily consumption per bar-brand pair
agg_df = df.groupby(['Date', 'Bar Name', 'Alcohol Type', 'Brand Name'])['Consumed (ml)'].sum().reset_index()


# Unique values for interface


In [ ]:
# Unique values for interface
bar_names = agg_df['Bar Name'].unique().tolist()
brand_names = agg_df['Brand Name'].unique().tolist()


# Forecasting and recommendation function


In [ ]:
# Forecasting and recommendation function
def forecast_inventory(bar_name, brand_name):
    group = agg_df[(agg_df['Bar Name'] == bar_name) & (agg_df['Brand Name'] == brand_name)]
    if group.empty:
        return f"No data available for {bar_name} - {brand_name}", None

    group = group.sort_values('Date')
    group.set_index('Date', inplace=True)
    date_range = pd.date_range(start=group.index.min(), end=group.index.max())
    group = group.reindex(date_range, fill_value=0)
    group.index.name = 'Date'

    if len(group) < 30:
        return f"Not enough data to forecast {bar_name} - {brand_name}", None

    train = group.iloc[:-14]
    test = group.iloc[-14:]

    try:
        fh = ForecastingHorizon(np.arange(1, 15), is_relative=True)
        model = AutoARIMA(sp=7, suppress_warnings=True)
        model.fit(train['Consumed (ml)'])
        forecast = model.predict(fh)

        rmse = np.sqrt(mean_squared_error(test['Consumed (ml)'], forecast))
        top_7_avg = train['Consumed (ml)'].nlargest(7).mean()
        safety_stock = train['Consumed (ml)'].std()
        recommended_par = np.ceil(top_7_avg + safety_stock)

        daily_stock = recommended_par * np.ones(len(test))
        remaining_stock = daily_stock - test['Consumed (ml)'].values
        stockout_days = (remaining_stock < 0).sum()
        overstock_avg = remaining_stock[remaining_stock > 0].mean() if any(remaining_stock > 0) else 0

        result_text = (
            f"Forecasting Results for {bar_name} - {brand_name}\n"
            f"RMSE: {rmse:.2f}\n"
            f"Recommended Par Level (ml): {recommended_par}\n"
            f"Stockout Days in Test Period: {stockout_days}\n"
            f"Average Overstock (ml): {overstock_avg:.2f}"
        )

        # Plot results
        plt.figure(figsize=(10, 5))
        plt.plot(test.index, test['Consumed (ml)'], label='Actual')
        plt.plot(test.index, forecast, label='Forecast')
        plt.axhline(y=recommended_par, color='red', linestyle='--', label='Par Level')
        plt.title(f"Forecast vs Actual for {bar_name} - {brand_name}")
        plt.xlabel("Date")
        plt.ylabel("Consumption (ml)")
        plt.legend()
        plt.tight_layout()

        return result_text, plt

    except Exception as e:
        return f"Forecasting failed for {bar_name} - {brand_name}: {str(e)}", None


In [ ]:
group = agg_df[(agg_df['Bar Name'] == "Anderson's Bar") & (agg_df['Brand Name'] == "Miller")]
print(group)


            Date        Bar Name Alcohol Type Brand Name  Consumed (ml)
0     2023-01-01  Anderson's Bar         Beer     Miller           0.00
194   2023-01-11  Anderson's Bar         Beer     Miller         484.08
357   2023-01-19  Anderson's Bar         Beer     Miller         462.30
634   2023-02-03  Anderson's Bar         Beer     Miller         618.55
704   2023-02-07  Anderson's Bar         Beer     Miller         398.77
827   2023-02-14  Anderson's Bar         Beer     Miller           0.00
1031  2023-02-25  Anderson's Bar         Beer     Miller         441.34
1068  2023-02-27  Anderson's Bar         Beer     Miller         572.00
1226  2023-03-08  Anderson's Bar         Beer     Miller           0.00
1318  2023-03-13  Anderson's Bar         Beer     Miller         443.41
1338  2023-03-14  Anderson's Bar         Beer     Miller         269.04
1375  2023-03-16  Anderson's Bar         Beer     Miller         489.24
1556  2023-03-26  Anderson's Bar         Beer     Miller        

In [ ]:
try:
    ...
except Exception as e:
    # handle the exception
    print(e)


In [ ]:
try:
    x = 1 / 0
except Exception as e:
    print("There was an error:", e)


There was an error: division by zero


In [ ]:


# Example time series data
y_train = pd.Series([100, 110, 115, 120, 125, 130, 135, 140])

try:
    model = AutoARIMA()
    model.fit(y_train)
    y_pred = model.predict(fh=[1, 2, 3])  # Forecast 3 steps ahead
except Exception as e:
    print("Forecasting failed:", e)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11

In [ ]:
try:
    model = AutoARIMA(...)
    model.fit(y_train)
    y_pred = model.predict(fh)
except Exception as e:
    print("Forecasting failed:", e)
import pandas as pd
from sktime.forecasting.arima import AutoARIMA


# Gradio Interface


In [ ]:
  # Gradio Interface
interface = gr.Interface(
    fn=forecast_inventory,
    inputs=[
        gr.Dropdown(choices=bar_names, label="Select Bar"),
        gr.Dropdown(choices=brand_names, label="Select Brand")
    ],
    outputs=[
        gr.Textbox(label="Summary"),
        gr.Plot(label="Forecast Plot")
    ],
    title="Hotel Bar Inventory Forecasting System",
    description="Select a bar and brand to view demand forecast and recommended inventory level."
)

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8259ca263e185dd632.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
